# Numerical Illustrations

This notebook reproduces the numerical studies in Section 5 of this [paper](https://arxiv.org/pdf/2301.00260.pdf) and Chapter 2.6 of this [dissertation](https://digital.lib.washington.edu/researchworks/bitstream/handle/1773/49764/Liu_washington_0250E_25117.pdf?sequence=1&isAllowed=y). If you find it useful, please cite

```
@inproceedings{liu2022likelihood,
  title={Likelihood Score under Generalized Self-Concordance},
  author={Liu, Lang and Harchaoui, Zaid},
  booktitle={NeurIPS 2022 Workshop on Score-Based Methods},
  year={2022}
}
```
and
```
@phdthesis{liu2022statistical,
  title={Statistical Divergences for Learning and Inference: Limit Laws and Non-Asymptotic Bounds},
  author={Liu, Lang},
  year={2022},
  school={University of Washington}
}
```

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Ellipse
from sklearn.linear_model import LogisticRegression

COLORS = plt.rcParams['axes.prop_cycle'].by_key()['color']

mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['legend.fontsize'] = 18
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['lines.markersize'] = 7.5
mpl.rcParams['text.usetex'] = False
mpl.rcParams["mathtext.fontset"] = 'cm'
plt.rcParams["font.family"] = "Times New Roman"

LINESTYLE = ['-', '--', '-.', (0, (1, 1)), '-', '--']
MARKER = ['8', 's', '', '', '^', '8']

## Shape of Confidence Set

In this section we illustrate the shape of the confidence set we considered on a logistic regression model.

In [ ]:
# generate data
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def generate_data(n, cov, theta0):
    X = np.random.multivariate_normal(np.zeros(2), cov, size=n)
    prob = sigmoid(-X @ theta0)
    Y = np.random.uniform(size=n)
    Y = (Y >= prob)*1
    Y = np.array((Y - 0.5)*2, dtype=int)
    return X, Y


# construct confidence set
def estimate(X, Y):
    model = LogisticRegression(random_state=0, fit_intercept=False).fit(X, Y)
    return model.coef_[0]


def conf_set(delta, X, theta_hat):
    prob = sigmoid(X @ theta_hat)
    emp_hess = X.T @ np.diag(prob * (1 - prob)) @ X / len(X)
    chol = np.linalg.cholesky(emp_hess)
    angle = np.linspace(0, 2*np.pi, 100)
    b = 0.15*np.sqrt(np.log(10/delta))
    w = np.vstack((b * np.cos(angle), b * np.sin(angle)))
    w = np.linalg.solve(chol, w).T
    return theta_hat + w


# compute losses
def single_loss(w0, w1, x, y):
    return np.log(1 + np.exp(-(x[0]*w0 + x[1]*w1)*y))


def loss(w0, w1, X, Y):
    losses = [single_loss(w0, w1, X[i], Y[i]) for i in range(n)]
    return np.mean(losses, axis=0)

In [ ]:
np.random.seed(51722)
n = 1000
theta0 = np.array([-1.0, 2.0])
covs = [np.array([[2, 0], [0, 1]]), np.array([[2, 1], [1, 1]]), np.array([[2, -1], [-1, 1]])]
levels = [[0.45, 0.5, 0.6, 0.75, 1.0], [0.55, 0.65, 0.85, 1.1, 1.4], [0.37, 0.4, 0.45, 0.55, 0.7]]
ellipses = [Ellipse([0, 0], 0.9, 1.95, angle=7, alpha=0.3, color=COLORS[1], label=r'$\delta = 0.95$'),
            Ellipse([0, 0], 0.68, 2.33, angle=9, alpha=0.3, color=COLORS[1], label=r'$\delta = 0.95$'),
            Ellipse([0, 0], 1.5, 2.55, angle=-6, alpha=0.3, color=COLORS[1], label=r'$\delta = 0.95$')]

# for loss contour
w0 = np.linspace(-3, 1, 100)
w1 = np.linspace(0, 4, 100)
W0, W1 = np.meshgrid(w0, w1)

nfig = len(covs)
fig, axes = plt.subplots(nrows=1, ncols=3, sharey=True)
fig.set_figheight(4)
fig.set_figwidth(12)
for k, cov in enumerate(covs):
    X, Y = generate_data(n, cov, theta0)
    theta_hat = estimate(X, Y)
    losses = loss(W0, W1, X, Y)
    conf = conf_set(0.95, X, theta_hat)
    ellipse = ellipses[k]
    ellipse.set(center=theta_hat)
    
    axes[k].tick_params(
        axis='both',
        which='both',
        bottom=False,
        top=False,
        left=False,
        labelbottom=False
    )
    
    axes[k].contour(W0, W1, losses, levels=levels[k], colors='gray')
    axes[k].plot([0], [0], label='Empirical risk contour', color='gray')
    axes[k].scatter(theta0[0], theta0[1], s=150, color=COLORS[3], marker='*', label=r'$\theta_0$')
    axes[k].scatter(theta_hat[0], theta_hat[1], s=150, color=COLORS[0], marker='*', label=r'$\theta_n$')
    axes[k].add_artist(ellipse)
    
handles, labels = axes[0].get_legend_handles_labels()
fig.tight_layout()
lgd = fig.legend(
    handles, labels, loc='lower center',
    bbox_to_anchor=(0.5, -0.14), ncol=4
)

## Estimation of the Effective Dimension

In this section we estimate the effective dimension from data and investigate its estimation error.

In [ ]:
# Least squares
def generate_least_square_data(n, cov, theta0):
    X = np.random.multivariate_normal(np.zeros(len(cov)), cov, size=n)
    Y = X @ theta0 + np.random.standard_normal(n)
    return X, Y


def ls_empirical_effective_dim(X, Y, theta_hat):
    res = X @ theta_hat - Y
    grad = res[:, np.newaxis] * X
    G_n = grad.T @ grad / len(Y)
    H_n = X.T @ X / len(Y)
    return np.trace(np.linalg.solve(H_n, G_n))


def ls_compute_abs_err(n, cov, theta0):
    X, Y = generate_least_square_data(n, cov, theta0)
    theta_hat = np.linalg.solve(X.T @ X, X.T @ Y)
    d_n = ls_empirical_effective_dim(X, Y, theta_hat)
    return np.abs(d_n / len(theta0) - 1)


# Logistic regression
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def logistic_estimate(X, Y):
    model = LogisticRegression(random_state=0, fit_intercept=False).fit(X, Y)
    return model.coef_[0]


def generate_logistic_data(n, cov, theta0, cond_prob):
    X = np.random.multivariate_normal(np.zeros(len(cov)), cov, size=n)
    prob = cond_prob(X, theta0)
    Y = np.random.uniform(size=n)
    Y = (Y <= prob)*1
    Y = np.array((Y - 0.5)*2, dtype=int)
    return X, Y


def logistic_empirical_effective_dim(X, Y, theta_hat):
    prob_hat = sigmoid(Y * (X @ theta_hat))
    grad = ((prob_hat - 1) * Y)[:, np.newaxis] * X
    G_n = grad.T @ grad / len(Y)
    hess_sqrt = np.sqrt(prob_hat * (1 - prob_hat))[:, np.newaxis] * X
    H_n = hess_sqrt.T @ hess_sqrt / len(Y)
    return np.trace(np.linalg.solve(H_n, G_n))


def logistic_effective_dim(X, cond_prob, theta0, theta):
    prob = cond_prob(X, theta0)
    prob_hat = sigmoid(X @ theta)
    grad = np.sqrt((1 - prob_hat)**2 * prob + prob_hat**2 * (1 - prob))[:, np.newaxis] * X
    G_star = grad.T @ grad / len(Y)
    hess_sqrt = np.sqrt(prob_hat * (1 - prob_hat))[:, np.newaxis] * X
    H_star = hess_sqrt.T @ hess_sqrt / len(Y)
    return np.trace(np.linalg.solve(H_star, G_star))


def cond_prob(X, theta0):
    return sigmoid(X @ theta0)


def logistic_compute_abs_err(n, cov, theta0):
    X, Y = generate_logistic_data(n, cov, theta0, cond_prob)
    theta_hat = logistic_estimate(X, Y)
    d_n = logistic_empirical_effective_dim(X, Y, theta_hat)
    return np.abs(d_n / len(theta0) - 1)


# Plot
def plot_abs_err(drange, nranges, abs_errs, repeat=10, fname=None, save=False):
    fig_prefix = '.'

    fig, axes = plt.subplots(nrows=1, ncols=2, sharey=True)
    fig.set_figheight(4.5)
    fig.set_figwidth(8)
    axes[0].set_ylabel(r'$\mathbb{E}| d_n / d_\star - 1 |$')
    axes[0].set_title(f'Least squares')
    axes[1].set_title(f'Logistic regression')

    for k, abs_err in enumerate(abs_errs):
        for i, d in enumerate(drange):
            err = np.array([e[0] for e in abs_err[i]])
            se = np.array([e[1]/np.sqrt(repeat) for e in abs_err[i]])
            axes[k].plot(
                nranges[k], err, label=f'd = {d}', color=COLORS[i],
                linestyle=LINESTYLE[i], marker=MARKER[i])
            axes[k].fill_between(nranges[k], err-se, err+se, color=COLORS[i], alpha=0.3)

        axes[k].set_xlabel('Sample Size')

    handles, labels = axes[0].get_legend_handles_labels()
    fig.tight_layout(rect=[0, 0.08, 1, 1])  # L, B, R, T
    lgd = fig.legend(
        handles, labels, loc='lower center',
        bbox_to_anchor=(0.5, -0.02), ncol=4)

    if save:
        fig.savefig(
            f'{fig_prefix}/{fname}.pdf',
            bbox_extra_artists=[lgd], bbox_inches='tight')

In [ ]:
ls_nrange = np.linspace(2000, 10000, 12, dtype=int)
logistic_nrange = np.linspace(2000, 10000, 12, dtype=int)
drange = [5, 10, 15, 20]
repeat = 30

ls_abs_err = []
logistic_abs_err = []
for d in drange:
    ls_err = []
    logistic_err = []
    cov = np.identity(d)
    theta0 = np.ones(d)
    for i, n in enumerate(ls_nrange):
        ls_tmp = []
        logistic_tmp = []
        for _ in range(repeat):
            ls_tmp.append(ls_compute_abs_err(n, cov, theta0))
            logistic_tmp.append(logistic_compute_abs_err(logistic_nrange[i], cov, theta0))
        ls_err.append((np.mean(ls_tmp), np.std(ls_tmp)))
        logistic_err.append((np.mean(logistic_tmp), np.std(logistic_tmp)))
    ls_abs_err.append(ls_err)
    logistic_abs_err.append(logistic_err)
    
plot_abs_err(drange, [ls_nrange, logistic_nrange], [ls_abs_err, logistic_abs_err], repeat=repeat,
             fname='est-effective-dim', save=False)